In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# -*- coding: utf-8 -*-
"""chexpert_noisfy.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Up66s8R2xK4yRqBX0sGYIZpyZ1ncOnhd
"""

#torch imports
import torch
torch.set_printoptions(profile="full")
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

#general imports
import os
import json
import pickle
import shutil
import datetime
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

x = '/content/gdrive/My\ Drive/chexpert_semantic_noise/noisy_data'
%cd $x
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/chexpert_semantic_noise/noisy_data
 acc_0.jpg				 negative_real_acc_0.jpg
'Atelectasis ACC_0.jpg'			 network_training_predictions
 baseline				 _old_images
 buffer					'Pleural Effusion ACC_0.jpg'
'Cardiomegaly ACC_0.jpg'		'Pleural Other ACC_0.jpg'
'Consolidation ACC_0.jpg'		'Pneumonia ACC_0.jpg'
'Edema ACC_0.jpg'			'Pneumothorax ACC_0.jpg'
'Enlarged Cardiomediastinum ACC_0.jpg'	 positive_acc_0.jpg
 forwardt				 positive_negative_acc_

In [2]:
!cat ./baseline/tgf_chexpert_utilities.py

cat: ./baseline/tgf_chexpert_utilities.py: No such file or directory


# INITIALIZE

In [3]:
import os
import sys
sys.path.append('./baseline')
import h5py
import time
import json
import pickle
import datetime
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
#torch imports
import torch
torch.set_printoptions(profile="full")
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
# some metrics
from sklearn.metrics import recall_score, precision_score, f1_score
# some local files
#sys.path.append('./baseline')
from tf_chexpert_utilities import *
from torch_chexpert_dataset import ChexpertDataset, MetricKeeper

In [5]:
def fpath(folder, noise):
    return '%s/n_%s'%(folder, str(noise))

"""## model compile func"""
def compile_model(model, binary=False):
  # Instantiate a logistic loss function that expects integer targets.
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  # Instantiate an accuracy metric.
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  if binary:
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.binary_accuracy
  
  # Instantiate an optimizer.
  optimizer = tf.keras.optimizers.Adam()
  # Instantiate some callbacks
  
  model.compile(optimizer=optimizer, loss=loss,metrics=[accuracy])
  return model

def initialize_logger(fname, train_length, val_length, test_length):
    if os.path.exists(fname):
      # if log file exists remove it permanately
      return h5py.File(fname, 'a')

    logger = h5py.File(fname, 'a')
    logger.create_dataset('train_loss', (train_length, ), compression='gzip')
    logger.create_dataset('train_real_loss', (train_length, ), compression='gzip')
    logger.create_dataset('train_acc', (train_length, ), compression='gzip')
    logger.create_dataset('train_real_acc', (train_length, ), compression='gzip')
    logger.create_dataset('train_precision', (train_length, ), compression='gzip')
    logger.create_dataset('train_recall', (train_length, ), compression='gzip')
    logger.create_dataset('train_f1_score', (train_length, ), compression='gzip')

    logger.create_dataset('val_loss', (train_length, ), compression='gzip')
    logger.create_dataset('val_real_loss', (train_length, ), compression='gzip')
    logger.create_dataset('val_acc', (train_length, ), compression='gzip')
    logger.create_dataset('val_real_acc', (train_length, ), compression='gzip')
    logger.create_dataset('val_precision', (train_length, ), compression='gzip')
    logger.create_dataset('val_recall', (train_length, ), compression='gzip')
    logger.create_dataset('val_f1_score', (train_length, ), compression='gzip')
    return logger

def iteration_log_metrics(KEY, iteration, loss, real_loss, acc, real_acc, precision, recall, _f1Score, 
                          positive_acc, positive_real_acc, negative_acc, negative_real_acc):
   
    if prediction_save_folder == None:
      print('unable to log iteration "prediction_save_folder" variable doesnt exist.')
      return
  
    save_ndarray(f'{prediction_save_folder}/{KEY}_loss_i_{iteration}', loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_loss_i_{iteration}', real_loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_acc_i_{iteration}', acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_acc_i_{iteration}', real_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_precision_i_{iteration}', precision)
    save_ndarray(f'{prediction_save_folder}/{KEY}_recall_i_{iteration}', recall)
    save_ndarray(f'{prediction_save_folder}/{KEY}_f1_score_i_{iteration}', _f1Score)

    save_ndarray(f'{prediction_save_folder}/{KEY}_positive_acc_i_{iteration}', positive_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_positive_real_acc_i_{iteration}', positive_real_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_negative_acc_i_{iteration}', negative_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_negative_real_acc_i_{iteration}', negative_real_acc)

def log_metric_keeper(metric_keeper, epoch):
  for name in metric_keeper.metric_name_list:
    is_negative_case = False
    for n in NEGATIVE_CASES:
      k = n.lower()
      k = k.replace(' ', '_').lower()
      if k in name:
        is_negative_case = True
        break
    if not is_negative_case:
      save_ndarray(f'{prediction_save_folder}/{name}_e_{epoch}',metric_keeper.get(name, epoch))
    else:
      save_ndarray(f'{prediction_save_folder}/{name}_e_{epoch}',metric_keeper.get_case(name, epoch))

def log_metrics(logger_file, KEY, EPOCH, loss, real_loss, acc, real_acc, precision, recall, _f1Score):
    save_ndarray(f'{prediction_save_folder}/{KEY}_loss_e_{EPOCH}', loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_loss_e_{EPOCH}', real_loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_acc_e_{EPOCH}', acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_acc_e_{EPOCH}', real_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_precision_e_{EPOCH}', precision)
    save_ndarray(f'{prediction_save_folder}/{KEY}_recall_e_{EPOCH}', recall)
    save_ndarray(f'{prediction_save_folder}/{KEY}_f1_score_e_{EPOCH}', _f1Score)

    #logger_file[f'{KEY}_loss'][EPOCH]       = loss
    #logger_file[f'{KEY}_real_loss'][EPOCH]  = real_loss
    #logger_file[f'{KEY}_acc'][EPOCH]        = acc
    #logger_file[f'{KEY}_real_acc'][EPOCH]   = real_acc
    #logger_file[f'{KEY}_precision'][EPOCH]  = precision
    #logger_file[f'{KEY}_recall'][EPOCH]     = recall
    #logger_file[f'{KEY}_f1_score'][EPOCH]   = _f1Score

"""## GET SAVED HDF5 DATASET FILE"""    
# get h5py DATASET file
dset = h5py.File('./buffer/baseline/dset.hdf5', 'r')
train_dataset = dset['train']
train_label_dataset = dset['train_label']
val_dataset = dset['val']
val_label_dataset = dset['val_label']
test_dataset = dset['test']
test_label_dataset = dset['test_label']

"""## SHUFFLING INDICES"""
# SHUFFLE TRAIN INDICES
np.random.seed(1234)
train_shuffled_idx_lst = np.arange(len(train_dataset))
np.random.shuffle(train_shuffled_idx_lst)
print("shuffled_train_indices: %s"%str(train_shuffled_idx_lst))
# SHUFFLE VAL INDICES
np.random.seed(5678)
val_shuffled_idx_lst = np.arange(len(val_dataset))
np.random.shuffle(val_shuffled_idx_lst)
print("shuffled_val_indices: %s"%str(val_shuffled_idx_lst))
# SHUFFLE TEST INDICES
np.random.seed(9012)
test_shuffled_idx_lst = np.arange(len(test_dataset))
np.random.shuffle(test_shuffled_idx_lst)
print("shuffled_test_indices: %s"%str(test_shuffled_idx_lst))
print('-'*30)

"""## GETTING THE GROUND TRUTH AND NOISY LABELS"""
train_noisy_label_dict  = {}
val_noisy_label_dict    = {}
train_ground_truth      = get_noisy_labels(0.0, 'train', len(train_dataset))
val_ground_truth        = get_noisy_labels(0.0, 'val', len(val_dataset))
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  - - - - - - - - 
noises = [0, 0.1, 0.2, 0.3, 0.4, 0.5, .60]
for n in noises:
  train_noisy_label_dict[n] = get_noisy_labels(n, 'train', len(train_dataset))
  val_noisy_label_dict[n]   = get_noisy_labels(n, 'val', len(val_dataset))


"""## SOME TRAINING PARAMATERS"""
# get custom datasets for network training

# GET SHUFFLED NAMES/LABES FOR TRAIN
train_ground_truth  = train_ground_truth[train_shuffled_idx_lst]
val_ground_truth    = val_ground_truth[val_shuffled_idx_lst]
for n in noises:
    train_noisy_label_dict[n]   =  train_noisy_label_dict[n][train_shuffled_idx_lst]
    val_noisy_label_dict[n]     =  val_noisy_label_dict[n][val_shuffled_idx_lst]

tensorboard_log_dir = './tensorboard_logs'
network_training_pred_folder = './network_training_predictions'
prediction_save_folder = './network_training_predictions/torch_baseline'
model_dir = './models'
model_save_dir = './models/torch_baseline'

make_sure_folder_exists(tensorboard_log_dir)
make_sure_folder_exists(network_training_pred_folder)
make_sure_folder_exists(prediction_save_folder)
make_sure_folder_exists(model_dir)
make_sure_folder_exists(model_save_dir)
make_sure_folder_exists('%s/%s'%(prediction_save_folder, '0'))
for n in noises:
    make_sure_folder_exists('%s/%i'%(prediction_save_folder, int(n*100)))

if (train_ground_truth == train_noisy_label_dict[0]).all():
  print('Train labels are correct')
else:
  print('Train labels are NOT CORRECT')

if (val_ground_truth == val_noisy_label_dict[0]).all():
  print('Validation labels are correct')
else:
  print('Validation labels are NOT CORRECT')

shuffled_train_indices: [14025 11750 13068 ... 23605  1318 25299]
shuffled_val_indices: [  93 1854 2937 ... 2366 2199 2164]
shuffled_test_indices: [ 831 3024 2751 ... 3302 2522 3314]
------------------------------
Given folder: ./tensorboard_logs EXISTS.
Given folder: ./network_training_predictions EXISTS.
Given folder: ./network_training_predictions/torch_baseline EXISTS.
Given folder: ./models EXISTS.
Given folder: ./models/torch_baseline EXISTS.
Given folder: ./network_training_predictions/torch_baseline/0 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/0 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/10 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/20 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/30 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/40 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/50 EXISTS.
Given folder: ./network_training_predictions/torch_b

# SET ARGS

In [6]:
# get args
# # # # if you are using a notebook comment the code below and uncomment the ' NOETBOOK MODE ' code block# # # #
#opt = get_args()

#BATCH_SIZE = opt.batch_size
#MAX_EPOCHS = opt.max_epoch
#NOISE_RATIO = opt.noise_ratio

#LR = opt.lr
#MOMENTUM = opt.momentum
#WEIGHT_DECAY = opt.weight_decay
#RESUME = opt.resume
skip_mean = True
# # # # NOTEBOOK MODE # # # #
BATCH_SIZE = 32
TRANSFORM_RESIZE = 224
MAX_EPOCHS = 50
NOISE_RATIO = 0
LR = 5e-3
LR_DECAY = 0.1
LR_DECAY_PER_EPOCH = 10
MOMENTUM = 0.9 
B=0.99
WEIGHT_DECAY = 1e-4
LOG_ITERATION_FACTOR = 100
RESUME = 0
#skip_mean = True
def get_str(item):
  lr_f  = len(str(LR)) - 2
  coeff = round(LR/(10**-lr_f))
  return f'[{coeff}e-{lr_f}]'

def get_checkpoint_fname():
  s_lr = f'lr_{get_str(LR)}'
  s_wd = f'wd_{get_str(WEIGHT_DECAY)}'
  s_b  = f'b_{BATCH_SIZE}' 
  return f'{s_lr}_{s_wd}_{s_b}_checkpoint.pth.tar'
# open h5py file to save the epoch states
#logger = initialize_logger('%s/%i/log.hdf5'%(prediction_save_folder, int(100*NOISE_RATIO)), len(train_dataset), len(val_dataset), len(test_dataset))

In [7]:
get_checkpoint_fname()

'lr_[5e-3]_wd_[5e-3]_b_32_checkpoint.pth.tar'

In [8]:
MAX_EPOCHS

50

# TRAIN

In [9]:
np_train_labal_dataset = np.array(train_label_dataset, dtype='object')[train_shuffled_idx_lst]
np_val_labal_dataset = np.array(val_label_dataset, dtype='object')[val_shuffled_idx_lst]
np_test_labal_dataset = np.array(test_label_dataset, dtype='object')[test_shuffled_idx_lst]

In [10]:
np_train_dataset = np.array(train_dataset, dtype=np.uint8)[train_shuffled_idx_lst]

In [11]:
np_val_dataset = np.array(val_dataset, dtype=np.uint8)[val_shuffled_idx_lst]

In [12]:
np_test_dataset = np.array(test_dataset, dtype=np.uint8)[test_shuffled_idx_lst]

In [13]:
if not skip_mean:
  # Get mean to initialize transform
  trainset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_train_dataset, noisy_labels=train_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=train_ground_truth, transform=transforms.ToTensor())
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
  mean = 0
  before = datetime.datetime.now()    
  for i, data in enumerate(trainloader, 0):
      imgs, labels, real_labels, index= data
      mean += torch.from_numpy(np.mean(np.asarray(imgs), axis=(2,3))).sum(0)
      if (i+1)%15 == 0:
          print_remaining_time(before, i+1, len(trainloader))
  mean = mean / len(trainset)
else:
  mean = [0.4365, 0.4365, 0.4365]
# get transform and initialize trainset and trainloader
# todo: normalize 0 to 1 for both training and testing
transform_train = transforms.Compose([transforms.Resize(TRANSFORM_RESIZE),
                                      transforms.RandomCrop(TRANSFORM_RESIZE, padding=4), 
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(),
                                      transforms.Normalize((mean[0], mean[1], mean[2]),(1.0, 1.0, 1.0))])
                                      
transform_test = transforms.Compose([transforms.Resize(TRANSFORM_RESIZE),
                                     transforms.ToTensor(),
                                     transforms.Normalize((mean[0], mean[1], mean[2]), (1.0, 1.0, 1.0))])


In [14]:
print(f'mean: {mean}')

mean: [0.4365, 0.4365, 0.4365]


In [15]:
trainset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_train_dataset, noisy_labels=train_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=train_ground_truth, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

valset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_val_dataset, noisy_labels=val_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=val_ground_truth, transform=transform_test)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f'trainset length: {len(trainset)}')
print(f'train loader length: {len(trainloader)}')
print(f'valset length: {len(valset)}')
print(f'val loader length: {len(valloader)}')
TRAINLOADER_LENGTH = len(trainloader)
VALLOADER_LENGTH = len(valloader)

trainset length: 26624
train loader length: 832
valset length: 3328
val loader length: 104


# LOAD AND TRAIN

In [ ]:
import torch
net = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
print(net)

In [16]:
NEGATIVE_CASES = ['Enlarged Cardiomediastinum', 
                  'Cardiomegaly', 
                  'Lung Opacity', 
                  'Lung Lesion',
                  'Edema',
                  'Consolidation',
                  'Pneumonia',
                  'Atelectasis',
                  'Pneumothorax',
                  'Pleural Effusion',
                  'Pleural Other',
                  'Fracture',
                  'Support Devices']

m_list = ['loss', 
          'acc', 
          'real_loss', 
          'real_acc', 
          'recall', 
          'precision', 
          'f_one_score',
          'positive_acc',
          'negative_acc',
          'positive_real_acc',
          'negative_real_acc']

# finish metric keeper

train_m_list = []
val_m_list = []

for m in m_list:
  train_m_list.append(f'train_{m}'.lower())
  val_m_list.append(f'val_{m}'.lower())
for n in NEGATIVE_CASES:
  k = n.lower()
  k = k.replace(' ', '_').lower()
  train_m_list.append(f'train_{k}_acc')
  train_m_list.append(f'train_{k}_real_acc')
  val_m_list.append(f'val_{k}_acc')
  val_m_list.append(f'val_{k}_real_acc')

# - //

train_metric_keeper = MetricKeeper(TRAINLOADER_LENGTH, MAX_EPOCHS, train_m_list)
val_metric_keeper = MetricKeeper(VALLOADER_LENGTH, MAX_EPOCHS, val_m_list)

for n in NEGATIVE_CASES:
  k = n.lower().replace(' ', '_').lower()
  # - #
  train_metric_keeper.reset_case(f'train_{k}_acc')
  train_metric_keeper.reset_case(f'train_{k}_real_acc')
  val_metric_keeper.reset_case(f'val_{k}_acc')
  val_metric_keeper.reset_case(f'val_{k}_real_acc')

train_metric_keeper.list_names()
val_metric_keeper.list_names()

METRIC KEEPER - METRIC NAMES: ['train_loss', 'train_acc', 'train_real_loss', 'train_real_acc', 'train_recall', 'train_precision', 'train_f_one_score', 'train_positive_acc', 'train_negative_acc', 'train_positive_real_acc', 'train_negative_real_acc', 'train_enlarged_cardiomediastinum_acc', 'train_enlarged_cardiomediastinum_real_acc', 'train_cardiomegaly_acc', 'train_cardiomegaly_real_acc', 'train_lung_opacity_acc', 'train_lung_opacity_real_acc', 'train_lung_lesion_acc', 'train_lung_lesion_real_acc', 'train_edema_acc', 'train_edema_real_acc', 'train_consolidation_acc', 'train_consolidation_real_acc', 'train_pneumonia_acc', 'train_pneumonia_real_acc', 'train_atelectasis_acc', 'train_atelectasis_real_acc', 'train_pneumothorax_acc', 'train_pneumothorax_real_acc', 'train_pleural_effusion_acc', 'train_pleural_effusion_real_acc', 'train_pleural_other_acc', 'train_pleural_other_real_acc', 'train_fracture_acc', 'train_fracture_real_acc', 'train_support_devices_acc', 'train_support_devices_real_ac

In [17]:
print(train_metric_keeper.get_case('train_enlarged_cardiomediastinum_acc', 0))
train_metric_keeper.get('train_acc',0)

nan


./baseline/torch_chexpert_dataset.py:95: RuntimeWarning: invalid value encountered in true_divide
  return np.divide(summed_values, summed_counts)


0.0

In [ ]:
print('************** SETUP PARAMETERS **************')
print(f'BATCH_SIZE:{BATCH_SIZE}')
print(f'TRANSFORM_RESIZE:{TRANSFORM_RESIZE}')
print(f'MAX_EPOCHS:{MAX_EPOCHS}')
print(f'NOISE_RATIO:{NOISE_RATIO}')
print(f'LR:{LR}')
print(f'LR_DECAY:{LR_DECAY}')
print(f'LR_DECAY_PER_EPOCH:{LR_DECAY_PER_EPOCH}')
print(f'MOMENTUM:{MOMENTUM}')
print(f'WEIGHT_DECAY:{WEIGHT_DECAY}')
print(f'LOG_ITERATION_FACTOR:{LOG_ITERATION_FACTOR}')
print(f'RESUME:{RESUME}')
print('**********************************************')

# get densenet
net = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)

# change the last classifier to predict 2 class instead of 10
num_ftrs = net.classifier.in_features
net.classifier = nn.Linear(in_features=num_ftrs, out_features=2, bias=True)

# get criterion
criterion = nn.CrossEntropyLoss()
#####--###############--###############--#####
net.cuda()
criterion.cuda()
#####--###############--###############--#####
optimizer = optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
num_classes = 2

if RESUME == 1:
  fn = os.path.join(model_save_dir, get_checkpoint_fname())
  ckpt = torch.load(fn)
  epoch_resume = ckpt['epoch']
  best_val_acc = ckpt['best_val_acc']
  net.load_state_dict(ckpt['state_dict'])
  optimizer.load_state_dict(ckpt['optimizer'])
  print('loading network SUCCESSFUL')
else:
  epoch_resume = 0
  best_val_acc = 0
  print('Starting training WITHOUT loading a network...') 

# TRAINING
for epoch in range(epoch_resume, MAX_EPOCHS):
  print('*'*20 + f'TRAIN e:{epoch}' + '*'*20)
  lr = LR * (LR_DECAY ** (epoch // LR_DECAY_PER_EPOCH))
  print(f'Learning rate of epoch:{epoch} is: {lr}')
  for param_group in optimizer.param_groups:
      param_group['lr'] = lr

  before_all = datetime.datetime.now()
  before = datetime.datetime.now()
  for i, data in enumerate(trainloader, 0):
    total_iteration = TRAINLOADER_LENGTH * epoch + i
    net.zero_grad()
    imgs, labels, real_ground_truths, index = data
    label_names = np_train_labal_dataset[index]
        # imgs               : data
        # labels             : noisy labels
        # real ground truths : not-noisy ground truths
        # index              : sample index
    imgs = Variable(imgs.cuda().float())
    labels = Variable(labels.cuda().long()[:,1])
    real_ground_truths = Variable(real_ground_truths.cuda().long())
    # forward
    logits = net(imgs.float())
    # get the accuracy and predictions of the current batch
    _, pred = torch.max(logits.data, -1)
    acc = float((pred==labels.data).sum()) 
    train_metric_keeper.add('train_acc', acc/BATCH_SIZE, total_iteration)
    # get indices
    positive_indices = np.where(labels.data.cpu() == 0)[0]
    negative_indices = np.where(labels.data.cpu() == 1)[0]
    positive_acc = float((pred[positive_indices]==labels.data[positive_indices]).sum())
    negative_acc = float((pred[negative_indices]==labels.data[negative_indices]).sum())
    train_metric_keeper.add('train_positive_acc', positive_acc/len(positive_indices), total_iteration)
    train_metric_keeper.add('train_negative_acc', negative_acc/len(negative_indices), total_iteration)
    # real acc
    real_acc = float((pred==real_ground_truths.data[:,1]).sum())
    train_metric_keeper.add('train_real_acc', real_acc/BATCH_SIZE, total_iteration)
    real_positive_indices = np.where(real_ground_truths.data[:,1].cpu() == 0)[0]
    real_negative_indices = np.where(real_ground_truths.data[:,1].cpu() == 1)[0]
    real_positive_acc = float((pred[positive_indices]==real_ground_truths.data[:,1][positive_indices]).sum())
    real_negative_acc = float((pred[negative_indices]==real_ground_truths.data[:,1][negative_indices]).sum())
    train_metric_keeper.add('train_positive_real_acc', real_positive_acc/len(real_positive_indices), total_iteration)
    train_metric_keeper.add('train_negative_real_acc', real_negative_acc/len(real_negative_indices), total_iteration)
    # negative cases
    for n in NEGATIVE_CASES:
      case_indices = np.where(label_names == n)[0]
      if len(case_indices) == 0:
        continue

      acc       = float((pred[case_indices]==labels.data[case_indices]).sum())
      real_acc  = float((pred[case_indices]==real_ground_truths.data[:,1][case_indices]).sum())
      k  = n.lower().replace(' ', '_').lower()      
      t  = f'train_{k}_acc'
      tt = f'train_{k}_real_acc'
      train_metric_keeper.add_case(t, acc/len(case_indices), total_iteration)
      train_metric_keeper.add_case(tt, real_acc/len(case_indices), total_iteration)

    # get current loss of the batch
    current_loss = criterion(logits, labels)
    individual_loss = current_loss.data
    train_metric_keeper.add('train_loss', individual_loss.cpu(), total_iteration)
    # real loss
    current_real_loss = criterion(logits, real_ground_truths[:,1])
    individual_real_loss = current_real_loss.data
    train_metric_keeper.add('train_real_loss', individual_real_loss.cpu(), total_iteration)
    # backward
    current_loss.backward()
    optimizer.step()
    # put prediction history in a list to save it later
    softmax_pred        = F.softmax(logits, -1).cpu().data
    #recall & prec & f1 score
    individual_recall = recall_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,    average="samples")
    individual_precision = precision_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5, average="samples")
    individual_f1_score = f1_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,        average="samples")
    # UPDATE METRIC KEEPER
    train_metric_keeper.add('train_recall', individual_recall, total_iteration)
    train_metric_keeper.add('train_precision', individual_precision, total_iteration)
    train_metric_keeper.add('train_f_one_score', individual_f1_score, total_iteration)    
    # LOGGING
    if (total_iteration+1) % LOG_ITERATION_FACTOR == 0:
      print(f'Logging train results of iteration: {i} on epoch:{epoch} with total itration: {total_iteration}')
      iteration_log_metrics(
        KEY='train', 
        iteration=total_iteration, 
        loss=individual_loss.cpu(),
        real_loss=individual_real_loss.cpu(),
        acc=acc/BATCH_SIZE,
        real_acc=real_acc/BATCH_SIZE,
        recall=individual_recall,
        precision=individual_precision,
        _f1Score = individual_f1_score,
        positive_acc = positive_acc/len(positive_indices),
        positive_real_acc = real_positive_acc/len(real_positive_indices),
        negative_acc = negative_acc/len(negative_indices),
        negative_real_acc = real_negative_acc/len(real_negative_indices))
      print_remaining_time(before, i+1, TRAINLOADER_LENGTH)

  print(f"TRAIN EPOCH END [{epoch}/{MAX_EPOCHS}]")
  for n in NEGATIVE_CASES:
    k  = n.lower().replace(' ', '_').lower()
    t   = f'train_{k}_acc'
    tt  = f'train_{k}_real_acc' 
    print(f"{t}: \t {train_metric_keeper.get_case(t,epoch)}")
    print(f"{tt}: \t {train_metric_keeper.get_case(tt,epoch)}")
  print(f"Loss: \t {train_metric_keeper.get('train_loss', epoch)}")
  print(f"REAL LOSS: \t {train_metric_keeper.get('train_real_loss', epoch)}")
  print(f"ACC: \t {train_metric_keeper.get('train_acc', epoch)}")
  print(f"Positive ACC: \t {train_metric_keeper.get('train_positive_acc', epoch)}")
  print(f"Negative ACC: \t {train_metric_keeper.get('train_negative_acc', epoch)}")
  print(f"REAL ACC: \t {train_metric_keeper.get('train_real_acc', epoch)}")
  print(f"Positive REAL ACC: \t {train_metric_keeper.get('train_positive_real_acc', epoch)}")
  print(f"Negative REAL ACC: \t {train_metric_keeper.get('train_negative_real_acc', epoch)}")
  print(f"RECALL: \t {train_metric_keeper.get('train_recall', epoch)}")
  print(f"PRECISION: \t {train_metric_keeper.get('train_precision', epoch)}")
  print(f"F1_SCORE: \t {train_metric_keeper.get('train_f_one_score', epoch)}")
  print('-'*100)
  #loss: %5f, real_loss: %5f, acc: %5f, real_acc:%5f, recall: %5f, precision: %5f, f1_score: %5f' %)
  print('*'*20+f'VALIDATION e:{epoch}'+'*'*20)
  # SAVE THE METRICS
  log_metric_keeper(train_metric_keeper, epoch)
  # ---------- VALIDATION STATE ----------
  net.eval()
  best_val_acc = 0
  before = datetime.datetime.now()
  with torch.no_grad():
    for i, data in enumerate(valloader, 0):
      total_iteration = VALLOADER_LENGTH * epoch + i
      # get data
      imgs, labels, real_ground_truths, index = data
      imgs = Variable(imgs.cuda())
      labels = Variable(labels.cuda().long()[:,1])
      label_names = np_val_labal_dataset[index]
      real_ground_truths = Variable(real_ground_truths.cuda().long())
      # LOSS and REAL LOSS
      logits = net(imgs.float())
      #current_loss
      loss = criterion(logits, labels).data
      val_metric_keeper.add('val_loss', loss.cpu(), total_iteration)
      real_loss = criterion(logits, real_ground_truths[:,1]).data
      val_metric_keeper.add('val_real_loss', real_loss.cpu(), total_iteration)
      # ACC
      _, pred = torch.max(logits.data, -1)
      acc = float((pred==labels.data).sum())
      val_metric_keeper.add('val_acc', acc/BATCH_SIZE, total_iteration)
      # POSITIVE ACC AND NEGATIVE ACC
      positive_indices = np.where(labels.data.cpu() == 0)[0]
      negative_indices = np.where(labels.data.cpu() == 1)[0]
      positive_acc = float((pred[positive_indices]==labels.data[positive_indices]).sum())
      negative_acc = float((pred[negative_indices]==labels.data[negative_indices]).sum())
      val_metric_keeper.add('val_positive_acc', positive_acc/len(positive_indices), total_iteration)
      val_metric_keeper.add('val_negative_acc', negative_acc/len(negative_indices), total_iteration)
      # REAL ACC
      real_acc = float((pred==real_ground_truths.data[:,1]).sum())
      val_metric_keeper.add('val_real_acc', real_acc/BATCH_SIZE, total_iteration)
      # POSITIVE REAL ACC AND NEGATIVE REAL ACC
      real_positive_indices = np.where(real_ground_truths.data[:,1].cpu() == 0)[0]
      real_negative_indices = np.where(real_ground_truths.data[:,1].cpu() == 1)[0]
      real_positive_acc = float((pred[positive_indices]==real_ground_truths.data[:,1][positive_indices]).sum())
      real_negative_acc = float((pred[negative_indices]==real_ground_truths.data[:,1][negative_indices]).sum())
      val_metric_keeper.add('val_positive_real_acc', real_positive_acc/len(real_positive_indices), total_iteration)
      val_metric_keeper.add('val_negative_real_acc', real_negative_acc/len(real_negative_indices), total_iteration)

      # negative cases
      for n in NEGATIVE_CASES:
        case_indices = np.where(label_names == n)[0]
        if len(case_indices) == 0:
          continue
        acc       = float((pred[case_indices]==labels.data[case_indices]).sum())
        real_acc  = float((pred[case_indices]==real_ground_truths.data[:,1][case_indices]).sum())
        k = n.lower().replace(' ', '_').lower()
        # save them _acc
        t  = f'val_{k}_acc'
        tt = f'val_{k}_real_acc'
        val_metric_keeper.add_case(t, acc/len(case_indices), total_iteration)
        val_metric_keeper.add_case(tt, real_acc/len(case_indices), total_iteration)

      # put prediction history in a list to save it later
      softmax_pred            = F.softmax(logits, -1).cpu().data
      # RECALL & prec & f1 score
      individual_recall = recall_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,    average="samples")
      individual_precision = precision_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5, average="samples")
      individual_f1_score = f1_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,        average="samples")
      # update metric
      val_metric_keeper.add('val_recall', individual_recall, total_iteration)
      val_metric_keeper.add('val_precision', individual_precision, total_iteration)
      val_metric_keeper.add('val_f_one_score', individual_f1_score, total_iteration)

      if (total_iteration+1) % LOG_ITERATION_FACTOR == 0:
        print(f'Logging VAL results of iteration: {i} on epoch:{epoch} with total itration: {total_iteration}')
        iteration_log_metrics(
          KEY='val', 
          iteration=total_iteration, 
          loss=loss.cpu(),
          real_loss=real_loss.cpu(),
          acc=acc/BATCH_SIZE,
          real_acc=real_acc/BATCH_SIZE,
          recall=individual_recall,
          precision=individual_precision,
          _f1Score = individual_f1_score,
          positive_acc = positive_acc/len(positive_indices),
          positive_real_acc = real_positive_acc/len(real_positive_indices),
          negative_acc = negative_acc/len(negative_indices),
          negative_real_acc = real_negative_acc/len(real_negative_indices))

        print_remaining_time(before, i+1, VALLOADER_LENGTH)

  ####################################################
  val_acc = val_metric_keeper.get('val_acc', epoch)
  is_best = val_acc > best_val_acc
  best_val_acc = max(val_acc, best_val_acc)
  # SAVE THE METRICS
  log_metric_keeper(val_metric_keeper, epoch)
  ####################################################
  print(f"VALIDATION EPOCH END [{epoch}/{MAX_EPOCHS}]")
  for n in NEGATIVE_CASES:
    k  = n.lower().replace(' ', '_').lower()
    t   = f'val_{k}_acc'
    tt  = f'val_{k}_real_acc' 
    print(f"{t}: \t {val_metric_keeper.get_case(t,epoch)}")
    print(f"{tt}: \t {val_metric_keeper.get_case(tt,epoch)}")
  print(f"Loss: \t {val_metric_keeper.get('val_loss', epoch)}")
  print(f"REAL LOSS: \t {val_metric_keeper.get('val_real_loss', epoch)}")
  print(f"ACC: \t {val_metric_keeper.get('val_acc', epoch)}")
  print(f"BEST ACC: \t {best_val_acc}")
  print(f"Positive ACC: \t {val_metric_keeper.get('val_positive_acc', epoch)}")
  print(f"Negative ACC: \t {val_metric_keeper.get('val_negative_acc', epoch)}")
  print(f"REAL ACC: \t {val_metric_keeper.get('val_real_acc', epoch)}")
  print(f"Positive REAL ACC: \t {val_metric_keeper.get('val_positive_real_acc', epoch)}")
  print(f"Negative REAL ACC: \t {val_metric_keeper.get('val_negative_real_acc', epoch)}")
  print(f"RECALL: \t {val_metric_keeper.get('val_recall', epoch)}")
  print(f"PRECISION: \t {val_metric_keeper.get('val_precision', epoch)}")
  print(f"F1_SCORE: \t {val_metric_keeper.get('val_f_one_score', epoch)}")
  print('-'*100)
  ####################################################
  net.train()
  ####################################################
  state = ({'epoch' : epoch,'state_dict' : net.state_dict(),'optimizer' : optimizer.state_dict(),'best_val_acc' : best_val_acc})

  fn = os.path.join(model_save_dir, get_checkpoint_fname())
  best_fn = os.path.join(model_save_dir, f'best_val_{get_checkpoint_fname()}')
  print(f'Saving model to: {fn}')
  torch.save(state, fn)
  # save state of the best validation    
  if is_best:
    print(f'Saving best val acc model to: {best_fn}')
    torch.save(state, best_fn)
  after = datetime.datetime.now()
  print(f'total time passed for the epoch {epoch}: {(after-before_all).seconds} seconds...')


************** SETUP PARAMETERS **************
BATCH_SIZE:32
TRANSFORM_RESIZE:224
MAX_EPOCHS:50
NOISE_RATIO:0
LR:0.005
LR_DECAY:0.1
LR_DECAY_PER_EPOCH:10
MOMENTUM:0.9
WEIGHT_DECAY:0.0001
LOG_ITERATION_FACTOR:100
RESUME:0
**********************************************


Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth



Starting training WITHOUT loading a network...
********************TRAIN e:0********************
Learning rate of epoch:0 is: 0.005
Logging train results of iteration: 99 on epoch:0 with total itration: 99
100/832(12.02%) finished. Estimated Remaining Time: 212.28 seconds.
Logging train results of iteration: 199 on epoch:0 with total itration: 199
200/832(24.04%) finished. Estimated Remaining Time: 180.12 seconds.
Logging train results of iteration: 299 on epoch:0 with total itration: 299
300/832(36.06%) finished. Estimated Remaining Time: 150.73 seconds.
Logging train results of iteration: 399 on epoch:0 with total itration: 399
400/832(48.08%) finished. Estimated Remaining Time: 122.04 seconds.
Logging train results of iteration: 499 on epoch:0 with total itration: 499
500/832(60.10%) finished. Estimated Remaining Time: 93.62 seconds.
Logging train results of iteration: 599 on epoch:0 with total itration: 599
600/832(72.12%) finished. Estimated Remaining Time: 64.96 seconds.
Logging

update 37


# pytorch playground

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

ValueError: ignored

In [ ]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_classes  =10
train_preds  	  = torch.zeros(len(trainset), num_classes) - 1.
num_hist = 10
train_preds_hist  = torch.zeros(len(trainset), num_hist, num_classes)
pl_ratio = 0.; nl_ratio = 1.-pl_ratio
train_losses      = torch.zeros(len(trainset)) - 1.
CC = 300
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    train_acc = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels, index = data
        #index = torch.ones(32) + i*32
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # get the predictions
        _, pred = torch.max(outputs.data, -1)      
        acc = float((pred==labels.data).sum())
        train_acc += acc
        # # # - # # # - # # #
        train_preds[index.cpu()] = F.softmax(outputs, -1).cpu().data
        train_losses[i] = loss.data
        # # # - # # # - # # #
        loss.backward()
        optimizer.step()
        # # # - # # # - # # #
        # print statistics
        running_loss += loss.item()
        if i % CC == CC - 1:    # print every CC mini-batches)
            print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss / CC, train_acc/(CC)))
            running_loss = 0.0
            train_acc = 0.0

print('Finished Training')

ValueError: ignored

# New Playground

In [ ]:
def channel_check(sample):
  for i in range(256):
    c = (sample[i][:,0] == sample[i][:,1]).all() and (sample[i][:,1] == sample[i][:,2]).all()
    if not c:
      return False
  
  return True

In [ ]:
length = len(np_train_dataset)
print_mod = 10
before = datetime.datetime.now()
for i in range(length):
  c = channel_check(np_train_dataset[i])
  if (i+1)%print_mod == 0:
    print('*'*20+f'{i}'+'*'*20)
    print_mod *= 2
    print_remaining_time(before, i+1, length)

  if not c:
    break

In [ ]:
c

In [ ]:
length = len(np_val_dataset)
print_mod = 10
before = datetime.datetime.now()
for i in range(length):
  c = channel_check(np_val_dataset[i])
  if (i+1)%print_mod == 0:
    print('*'*20+f'{i}'+'*'*20)
    print_mod *= 2
    print_remaining_time(before, i+1, length)
    
  if not c:

    break
print(c)

In [ ]:
length = len(np_test_dataset)
print_mod = 10
before = datetime.datetime.now()
for i in range(length):
  c = channel_check(np_test_dataset[i])
  if (i+1)%print_mod == 0:
    print('*'*20+f'{i}'+'*'*20)
    print_mod *= 2
    print_remaining_time(before, i+1, length)

  if not c:
    break
print(c)

********************9********************
10/3356(0.30%) finished. Estimated Remaining Time: 0.00 seconds.
********************19********************
20/3356(0.60%) finished. Estimated Remaining Time: 0.00 seconds.
********************39********************
40/3356(1.19%) finished. Estimated Remaining Time: 0.00 seconds.
********************79********************
80/3356(2.38%) finished. Estimated Remaining Time: 0.00 seconds.
********************159********************
160/3356(4.77%) finished. Estimated Remaining Time: 0.00 seconds.
********************319********************
320/3356(9.54%) finished. Estimated Remaining Time: 0.00 seconds.
********************639********************
640/3356(19.07%) finished. Estimated Remaining Time: 4.24 seconds.
********************1279********************
1280/3356(38.14%) finished. Estimated Remaining Time: 3.24 seconds.
********************2559********************
2560/3356(76.28%) finished. Estimated Remaining Time: 1.55 seconds.
True


In [ ]:
a = np_test_dataset[0] - 2

NameError: ignored